<a href="https://colab.research.google.com/github/yukinaga/object_detection/blob/main/section_4/01_detr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DETRによる物体検出
PyTorchを使って、DETRによる物体検出を実装します。  
なお、このノートブックのコードはFacebook Researchが用意した以下のサンプルコードを参考にしています。  
https://colab.research.google.com/github/facebookresearch/detr/blob/colab/notebooks/detr_attention.ipynb

## 設定
必要なライブラリの導入、各物体名が格納されたリストの用意、必要な関数の定義を行います。

In [ ]:
%config InlineBackend.figure_format = "retina"  # 画像の解像度を向上

import matplotlib.pyplot as plt
from PIL import Image

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
import torchvision.transforms as T

torch.set_grad_enabled(False)  # 訓練は行わないので勾配の計算は不要

# データセットCOCOの物体名
names = [
    "N/A", "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", 
    "truck", "boat", "traffic light", "fire hydrant", "N/A","stop sign", "parking meter",
    "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
    "giraffe", "N/A", "backpack","umbrella", "N/A", "N/A", "handbag", "tie",
    "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "N/A",
    "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut",
    "cake", "chair", "couch", "potted plant", "bed", "N/A", "dining table",
    "N/A", "N/A", "toilet", "N/A", "tv", "laptop", "mouse", "remote", "keyboard",
    "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "N/A",
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# バウンディングボックスの座標変換
def cxcywh_to_4corners(x):
    x_c, y_c, w, h = x.unbind(1)
    box = [(x_c - 0.5 * w),
           (y_c - 0.5 * h),
           (x_c + 0.5 * w),
           (y_c + 0.5 * h)]
    return torch.stack(box, dim=1)

# バウンディングボックスのスケール変換
def fit_boxes(y_box, size):
    w, h = size
    box = cxcywh_to_4corners(y_box)
    box = box * torch.tensor([w, h, w, h], dtype=torch.float)
    return box

# 結果の表示
def show_results(img, ps, boxes):

    boxes = boxes.tolist()

    plt.figure(figsize=(16,10))
    plt.imshow(img)

    ax = plt.gca()
    for p, (x_min, y_min, x_max, y_max) in zip(ps, boxes):
        ax.add_patch(plt.Rectangle((x_min, y_min),
                                   x_max - x_min,
                                   y_max - y_min,
                                   fill=False,
                                   color="red",
                                   linewidth=3))
        
        result_id = p.argmax()
        label = f"{names[result_id]}: {p[result_id]:0.3f}"
        ax.text(
            x_min,y_min,
            label, fontsize=12,
            bbox=dict(facecolor="orange", alpha=0.4)
            )
        
    plt.axis("off")
    plt.show()

## モデルの読み込み
PyTorch Hubを使い、モデル「DETR-R50」を読み込みます。  
https://github.com/facebookresearch/detr#model-zoo

In [ ]:
model = torch.hub.load("facebookresearch/detr", "detr_resnet50", pretrained=True)
model.eval()  # 評価モード
print(model)

## 画像の読み込み
手元の画像をアップロードし、読み込みます。

In [ ]:
from google.colab import files

uploaded_files = files.upload()  # ファイルのアップロード
img_origin = Image.open(next(iter(uploaded_files)))

## モデルを使った予測
訓練済みのモデルを使い、物体の位置と種類を予測します。

In [ ]:
# 画像の変換
transform = T.Compose([
    T.Resize(800),  # 短い辺を800に変換
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 標準化
])
x = transform(img_origin).unsqueeze(0)  # unsqueezeでバッチ対応

# 予測
y = model(x)

# 予測結果の選別
ps = y["pred_logits"].softmax(-1)[0, :, :-1]
extracted = ps.max(-1).values > 0.95 # 0.95より確率が大きいものを選別

# バウンディングボックスの座標計算
boxes = fit_boxes(y["pred_boxes"][0, extracted], img_origin.size)

# 予測結果の表示
show_results(img_origin, ps[extracted], boxes)

## Attention weightの可視化
Decoderの最後の層のattention weightを可視化します。  
モデルが、バウンディングボックスと物体の種類を予測するために注目している箇所を確かめます。  

In [ ]:
# 各層の出力を格納するリスト
conv_ys = []
dec_attn_weights = []

# 順伝播時に各層で行う処理
hooks = [
    model.backbone[-2].register_forward_hook(
        lambda self, x, y: conv_ys.append(y)
    ),
    model.transformer.decoder.layers[-1].multihead_attn.register_forward_hook(
        lambda self, x, y: dec_attn_weights.append(y[1])
    )
]

# 順伝播
y = model(x)

# 特徴マップの幅と高さを取得
h, w = conv_ys[0]["0"].tensors.shape[-2:]

# Attentionの表示
fig, axes = plt.subplots(ncols=len(boxes), nrows=2, figsize=(22, 7))
for i, axs, (x_min, y_min, x_max, y_max) in zip(extracted.nonzero(), axes.T, boxes):
    ax = axs[0]
    ax.imshow(dec_attn_weights[0][0, i].view(h, w))
    ax.axis("off")
    
    ax = axs[1]
    ax.imshow(img_origin)
    ax.add_patch(plt.Rectangle(
        (x_min, y_min),
        x_max - x_min,
        y_max - y_min,
        fill=False, color="blue", linewidth=2)
    )
    ax.axis("off")
    ax.set_title(names[ps[i].argmax()])

fig.tight_layout()